# 支持向量机 Support Vector Machines

本节包括以下内容：
1. 间隔 Margin
2. 记号 Notation
3. 函数间隔和几何间隔 Functional and geometric margins
4. 最大间隔分类器 The optimal margin classifier
5. 拉格朗日对偶性 Lagrange duality
6. 最大间隔分类器 Optimal margin classifier
7. 核方法 Kernels
8. 正则化、不可分的情况 Regularization and the non-separable case
9. SMO算法 The SMO algorithm
10. SMO


### 1. 间隔 Margin

回忆逻辑回归，概率 $p(y=1|x; \theta)$ 由 $h_\theta(x) = g(\theta^Tx)$ 计算而得。如果 $h_\theta(x) \geq 0.5$，则我们预测输入 $x$ 为正类，换言之，当且仅当 $\theta^Tx \geq 0$ 时，模型预测 $x$ 为正类。并且，根据我们的概率假设，$\theta^Tx$ 越大，$x$ 取正类的概率就越大，或者说，我们对于预测 $x$ 为正类的信心就越大。

也就是说，如果 $\theta^Tx \gg 0$，那么我们可以非常自信地预测 $y=1$。类似的，如果 $\theta^Tx \ll 0$，则我们可以非常自信地预测 $y=0$。直觉上看，如果我们的成本函数考虑这一点，那么我们可以得到一个不错的分类器。对此，稍后我们将会引入函数间隔的概念。

换一种思路来看，$\theta^Tx=0$ 代表决策边界（也称为**分割超平面 separating hyperplane**），离决策边界越远的点，我们对准确预测其分类的信心越大。直觉上看，在数据集线性可分的前提下，在所有决策边界集合中，使得样本点尽可能远离的决策边界，通常代表着更好的模型。对此，稍后我们会引入几何间隔的概念。

### 2. 记号 Notation

对于支持向量机，我们将引入和广义线性模型所不同的符号来阐述二元分类问题。给定标签 $y$ 和特征 $x$，令 $y \in \{-1, 1\}$（而不是 $\{0, 1\}$），同时我们不再使用 $\theta$ 作为参数向量，而是使用 $w, b$ 来描述分类器
$$ h_{w, b}(x) = g(w^Tx + b) $$

这里，当 $z \geq 0$ 时，$g(z)=1$，否则 $g(z) = -1$。$w, b$ 的记号，使我们将截距项 $b$ 从其它参数独立出来，从而我们也不需要再假定 $x_0=1$，因而这里 $x$ 是 $n$ 维向量，而不是 $n+1$ 维向量。也就是说，$b$ 就是之前的 $\theta_0$，而 $w$ 是之前的 $[\theta_1...\theta_n]^T$

注意到，根据我们对 $g$ 的上述定义，我们的分类器将直接预测 $1$ 或 $-1$（类比感知器），而不再需要和逻辑回归一样，必须包含预测 $y=1$ 的概率这个中间步骤。

### 3. 函数间隔和几何间隔 Functional and Geometric Margins

给定一个训练样本 $(x^{(i)}, y^{(i)})$，定义该训练样本 $(w, b)$ 的**函数间隔 functional margin**为
$$ \hat{\gamma}^{(i)} = y^{(i)}(w^Tx+b) $$

注意到，如果 $y^{(i)}=1$，要使函数间隔更大（也即使我们的预测更有信心更准确），则需要 $w^Tx+b$ 是一个非常大的正数。相反，如果 $y^{(i)}=-1$，要使函数间隔更大，则需要 $w^Tx+b$ 是一个（绝对值）非常大的负数。另外，当 $y^{(i)}(w^Tx+b) > 0$ 时，我们对这个训练样本的预测是准确的。因此，一个非常大的函数间隔，同时代表着高置信度以及正确的分类。

对于上述给定的线性分类器 $g$ （取值范围为 $\{-1, 1\}$），函数间隔的一个特性使其并不能成为置信度的很好测量指标。考虑如下情况：将 $w$ 替换为 $2w$，$b$ 替换为 $2b$，由于 $g(w^Tx+b)=g(2w^Tx+2b)$，分类结果 $h_{w,b}(x)$ 将保持不变。然而，这个时候函数间隔扩大了两倍。换言之，我们可以同比例地扩大参数 $w, b$，从而以任意倍数扩大函数间隔，但这个操作对分类器本身的效果而言，不带来任何变化。从直觉上看，我们应该对参数做归一化操作，使得 $||w||_2=1$，替换 $(w,b)$ 为 $(\frac{w}{||w||_2}, \frac{b}{||w||_2})$，在此基础上，再计算函数间隔。之后我们将会回到这个问题。

给定一个训练集，我们定义训练集 $S$ 的函数间隔为所有单个训练样本的函数间隔的最小值，即
$$ \hat{\gamma} = \min_{i=1,...,m}\hat{\gamma}^{(i)} $$

接下来，考虑**几何间隔 geometric margin**

注意到，向量 $w$ 和分割超平面总是正交的，考虑处于正类的点 $A$，其特征为 $x^{(i)}$，标签为 $y^{(i)}=1$，假设点 $A$ 到决策边界的距离为 $\gamma^{(i)}$，相交点为 $B$。

这里 $\frac{w}{||w||}$ 是 $w$ 方向的单位向量，点 $A$ 可以用向量 $x^{(i)}$ 表示。因此点 $B$ 就是 $x^{(i)} - \gamma^{(i)} \cdot \frac{w}{||w||}$，由于点 $B$ 处于决策边界上，满足 $w^Tx+b=0$，因此
$$ w^T(x^{(i)}-\gamma^{(i)}\frac{w}{||w||})+b=0 $$
解方程得
$$ \gamma^{(i)} = \frac{w^Tx^{(i)}+b}{||w||} = (\frac{w}{||w||})^Tx^{(i)} + \frac{b}{||w||} $$

这是正类的情况，对于正类和反类的情况
$$ \gamma^{(i)} = y^{(i)}((\frac{w}{||w||})^Tx^{(i)} + \frac{b}{||w||}) $$

注意到，$\frac{\hat{\gamma}}{||w||}=\gamma$，当 $||w||=1$ 时，函数间隔等于几何间隔。并且，几何间隔不会受到参数扩大缩小的影响。也即是说，我们可以按任意约束对 $w, b$ 进行缩放，而不改变几何间隔。

最后，给定一个训练集，我们同样定义训练集 $S$ 的几何间隔为所有单个训练样本的几何间隔的最小值，即
$$ \gamma = \min_{i=1,...,m}\gamma^{(i)} $$

### 4. 最大间隔分类器 The Optimal Margin Classifier

根据上述对间隔的讨论，我们希望找到一条最大化（几何）间隔的决策边界，这样的决策边界同时保证了分类的正确性和置信度，它会尽可能地远离正类和反类。

这里，我们暂且假定数据集线性可分，为了最大化几何间隔，我们引入以下优化问题：
$$
\begin{split}
& \max_{\gamma, w, b} \gamma \\
& s.t. y^{(i)}\frac{w^Tx^{(i)}+b}{||w||} = y^{(i)}(w^Tx^{(i)}+b) \geq \gamma, i=1,...,m \\
& ||w||=1
\end{split}
$$

也就是说，给定约束条件 $||w||=1$ （从而函数间隔等于几何间隔），且所有训练样本的函数间隔大于等于 $\gamma$，也即几何间隔大于等于 $\gamma$，我们要想找到最大的 $\gamma$。解这个优化问题求得的 $(w, b)$ 就保证了针对训练集的最大几何间隔。

如果上述优化问题可解，则我们已经获得了最大间隔分类器。但由于 $||w||=1$ 是一个非凸约束，标准的优化软件无法对其求解。因此，我们试着对优化问题做一定的转换：
$$
\begin{split}
& \max_{\gamma, w, b} \frac{\hat{\gamma}}{||w||} \\
& s.t. y^{(i)}(w^Tx^{(i)}+b) \geq \hat{\gamma}, i=1,...,m \\
\end{split}
$$

这里，我们的优化目前是函数间隔除以权重的模，根据之前的定义，等于几何间隔，也即优化目标不变。而约束条件为所有训练样本的函数间隔大于等于 $\hat{\gamma}$。这样，我们去除了 $||w||=1$ 的非凸约束，但是得到了非凸优化目标。目前依然没有求解的现成方法。

我们需要对这个优化问题做进一步转换。注意上一节提到过，我们可以按任意约束对 $w, b$ 进行缩放，而不改变几何间隔（也就不改变模型，但是会改变函数间隔）。引入约束函数间隔等于 $1$
$$ \hat{\gamma}=1 $$
从而，上面的优化目标 $\frac{\hat{\gamma}}{||w||}=\frac{1}{||w||}$，而最大化 $\frac{1}{||w||}$ 等价于最小化 $||w||^2$，于是我们得到下列优化问题：
$$
\begin{split}
& \min_{\gamma,w,b}\frac{1}{2}||w||^2 \\
& s.t. y^{(i)}(w^Tx^{(i)}+b) \geq 1, i=1,...,m \\
\end{split}
$$

这个问题是线性约束下凸二次函数的优化，利用一些二次规划（Quadratric Programming）的软件，可以有效求解。求解得到的结果，就是最大间隔分类器。（由于有约束条件的存在，通常不会用梯度下降类似的方法来解条件极值的问题）

### 5. 拉格朗日对偶性

为了从凸优化的角度来理解最大间隔分类器，我们先介绍条件极值的问题。考虑以下问题：
$$
\begin{split}
& \min_{w}f(w) \\
& s.t. \, h_i(w)=0, i=1,...,l \\
\end{split}
$$
这个问题在多元函数微分学中，通常使用拉格朗日乘数法求解，上面这个问题对应的**拉格朗日函数 Lagrangian**为
$$ L(w,b) = f(w) + \sum_{i=1}^l \beta_i h_i(w) $$
$\beta_i$ 被称为拉格朗日乘子，使拉格朗日函数的偏导为0：
$$ \frac{\partial L}{\partial w_i} = 0; \, \frac{\partial L}{\partial \beta_i}=0 $$
求解 $w$ 和 $\beta$ 即可。

本节，我们将推广条件极值求解问题到约束条件允许包含不等式的情况。考虑以下优化问题，我们称之为**原问题 primal problem**：
$$
\begin{split}
& \min_{w}f(w) \\
& s.t. \, g_i(w) \leq 0, i=1,...,k \\
& h_i(w)=0, i=1,...,l \\
\end{split}
$$
我们定义**广义拉格朗日函数 Generalized Lagrangian**
$$ L(w,\alpha,\beta) = f(w) + \sum_{i=1}^k \alpha_i g_i(w) + \sum_{i=1}^l \beta_i h_i(w) $$
这里，$\alpha_i$ 和 $\beta_i$ 为拉格朗日乘子。考虑下面这个值
$$ \theta_p(w) = \max_{\alpha, \beta: \alpha_i \geq 0}L(w,\alpha,\beta)$$
当给定 $w$ 时，如果 $w$ 没能满足原问题的约束（比如对某个 $i$， $g_i(w)>0$ 或 $h_i(w) \neq 0$），那么
$$
\begin{split}
\theta_p(w) &= \max_{\alpha, \beta: \alpha_i \geq 0}f(w) + \sum_i^k \alpha_i g_i(w) + \sum_i^l \beta_i h_i(w) \\
&= \infty
\end{split}
$$
相反，当约束条件均满足时，$\theta_p(w) = f(w)$，所以
$$ \theta_p(w) = \left\{
\begin{aligned}
& f(w) &如果w满足约束 \\
& \infty & w不满足约束
\end{aligned}
\right.
$$
也就是说，对于所有满足约束的 $w$ 来说，$\theta_p$ 的值和我们目标函数的值一致，否则就取正无穷。从而，考虑下面这个最优化问题
$$ \min_w \theta_p(w) = \min_w {\max_{\alpha,\beta:\alpha \geq 0}L(w,\alpha,\beta)} $$
易知这个问题和原问题是相同的。我们将该问题的最优值记为 $p^* = \min_w \theta_p(w)$，也就是原问题的最优值。

接下来我们看一个不同的问题。定义
$$ \theta_D(\alpha, \beta) = \min_w L(w,\alpha,\beta) $$
对于 $\theta_p$，我们优化的参数是 $\alpha, \beta$，而对于 $\theta_D$，我们优化的参数是 $w$。于是我们可以提出这个**对偶问题 dual problem**：
$$ \max_{\alpha,\beta: \alpha_i \geq 0}\theta_D(\alpha,\beta) = \max_{\alpha,\beta: \alpha_i \geq 0} \min_w L(w,\alpha, \beta) $$

除去求最大值和最小值的顺序之外，原问题和对偶问题是完全相同的。我们假定对偶问题的最优值为 $d^* = \max_{\alpha,\beta: \alpha_i \geq 0} \theta_D(w)$，易知：
$$ d^* = \max_{\alpha,\beta: \alpha_i \geq 0} \min_w L(w,\alpha, \beta) \leq \min_w {\max_{\alpha,\beta:\alpha \geq 0}L(w,\alpha,\beta)} = p^* $$
在某种条件满足的情况下，有
$$ d^* = p^* $$
这时，我们可以通过解对偶问题来解原问题。

而原问题和对偶问题相等的条件为：假设 $f$ 和 $g_i$ 都是凸函数，$h_i$ 远交（affine），$g_i$ 是可行约束。

在上述条件满足时，必定存在 $w^*, \alpha^*, \beta^*$ 使得 $w^*$ 是原问题的解，$\alpha^*, \beta^*$ 是对偶问题的解，同时 $p^* = d^* = L(w^*, \alpha^*, \beta^*)$。并且，$w^*, \alpha^*, \beta^*$ 满足 **KKT条件 Karush-Kuhn-Tucker条件**：
$$
\begin{split}
\frac{\partial}{\partial w_i} L(w^*, \alpha^*, \beta^*) &= 0, i=1,...,n \\
\frac{\partial}{\partial \beta_i} L(w^*, \alpha^*, \beta^*) &= 0, i=1,...,l \\
\alpha^*_i g_i(w^*) &= 0, i=1,...,k \\
g_i(w^*) &\leq 0,i=1,...,k \\
\alpha^* &\geq 0,i=1,...,k
\end{split}
$$
逆命题也是成立的，如果 $w^*, \alpha^*, \beta^*$ 满足KKT条件，则它们一定分别是原问题和对偶问题的解。

$\alpha^*_i g_i(w^*) = 0$ 也称为**对偶互补约束 dual complementarity condition**，这个约束满足意味着，如果 $\alpha_i^* > 0$，则 $g_i(w^*) = 0$。这个特性后面会用于证实，支持向量机只有少数几个支持向量。当介绍SMO算法时，对偶互补约束也会用于给出收敛测试。